In [2]:
import pandas as pd 


In [3]:
df=pd.read_csv('Combined Data.csv')


Так как большинство датасетов по моей специальности уже рамечены, я удалила столбец с разметкой, а также перемешала строки. 100 строк я сохранила в датасет для ручной разметки, сохранив его в файл 'MyParseData'. Еще 200 строк сохранила для разметки по правилам. Изначально в датасете была разметка по психическим расстройствам, но в данном случае разделим данные по наличию и отсуствию суицидальной готовности. 

In [4]:
del df['status']
del df['Unnamed: 0']

Датасет был перемешан, так как изначальные данные были уже отсортированы и первые строки принадлежали одному кластеру

In [7]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,statement
0,I'm so nervous I feel why
1,I have very little understanding of developing...
2,duncanmacrae hmmm well good luck with that
3,I was working at a labor pool when ever I coul...
4,I tried to hang myself and it happened like th...


In [51]:
myparse = df.iloc[:100]
myparse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   statement  100 non-null    object
dtypes: object(1)
memory usage: 928.0+ bytes


In [52]:
myparse.to_csv('MyParseData.csv')

In [11]:
rulesparse=df.iloc[100:300]
rulesparse.head()

,statement
100,jonathanrknight you sure did do some tweet and...
101,is just loving the fact that it is 0 in the a ...
102,mother depression arc
103,"I am 25 years old, male, will not disclose my ..."
104,please do not reply to this ill just have to g...


In [75]:

myparse=pd.read_csv('MyParseData.csv')
myparse.head()

,statement,status
0,is at beso in hollywood so tired,non-suicidal
1,Unsual sweating \nSo i have this unsual sweati...,non-suicidal
2,I got innocently bit by a feral kitten trapped...,non-suicidal
3,Anyone get this stress induced symptom? Every ...,non-suicidal
4,"I bought a shirt for 200, but I've never worn ...",non-suicidal


In [46]:
rulesparse['statement']=rulesparse['statement'].fillna('neutral')
rulesparse.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 100 to 299
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   statement  200 non-null    string
dtypes: string(1)
memory usage: 1.7 KB


C:\Users\767A~1\AppData\Local\Temp/ipykernel_1880/1514207645.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rulesparse['statement']=rulesparse['statement'].fillna('neutral')


In [67]:
rulesparse.to_excel('Rulesparse.xlsx')

Для ручной разметки я выбрала LabelStudio. Возможно, для распознавания текста лучше было использовать tagtog, но в сети были комментарии о том, что у него хуже интерфейс, а преимуществ не очень много. Возможно, когда моя задача будет не учебной, а практической, я подробнее изучу этот инструмент. 

In [23]:
import numpy as np
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to C:\Users\Анастасия
[nltk_data]     Рахманина\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [63]:
#Определим слова-синонимы для разных видов ментальных проблем
suicidal=['die','kill','murder','end','suicide','suicidal','goodbye']


In [68]:
def get_status(text):
    words=nltk.word_tokenize(text.lower())
    num_anxious = sum([1 for word in words if word in anxious_words])
    num_depressive = sum([1 for word in words if word in depressive_words])
    num_psycho = sum([1 for word in words if word in psycho_words])
    num_bo = sum([1 for word in words if word in burned_out])
    num_suicidal = sum([1 for word in words if word in suicidal])
    if num_suicidal>1:
        return 'suicidal'
    else:
        return 'non-suicidal'

    
        
    
    


In [69]:
rulesparse['status']=rulesparse['statement'].apply(get_status)

C:\Users\767A~1\AppData\Local\Temp/ipykernel_1880/3703138813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rulesparse['status']=rulesparse['statement'].apply(get_status)


In [70]:
rulesparse.head()

,statement,status
100,jonathanrknight you sure did do some tweet and...,non-suicidal
101,is just loving the fact that it is 0 in the a ...,non-suicidal
102,mother depression arc,non-suicidal
103,"I am 25 years old, male, will not disclose my ...",suicidal
104,please do not reply to this ill just have to g...,non-suicidal


Объединим датасеты

In [139]:
merged_df=pd.concat([myparse,rulesparse])

In [140]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
merged_df['status'] = le.fit_transform(merged_df['status'].astype(object))
merged_df = merged_df.sample(frac=1).reset_index(drop=True)


Так как в программе уроки по машинному обучению были открыты недавно и я их еще не освоила, использую код из урока. 

In [176]:
train_df = merged_df[0:150]
test_df=merged_df[150:]


In [157]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


In [177]:
labeled_texts, unlabeled_texts = train_test_split(train_df, train_size=0.5, random_state=42)

In [178]:
def train_model(labeled_texts):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(labeled_texts['statement'])
    y = labeled_texts['status']

    # Обучение модели логистической регрессии на размеченных данных
    model = LogisticRegression()
    model.fit(X, y)

    return model, vectorizer


In [179]:
model, vectorizer = train_model(labeled_texts)

In [180]:
X_unlabeled = vectorizer.transform(unlabeled_texts['statement'])
y_unlabeled_predicted = model.predict(X_unlabeled)

In [181]:
model, vectorizer = train_model(labeled_texts)

In [182]:
X_test = vectorizer.transform(test_df['statement'])
y_test_predicted = model.predict(X_test)
f1 = f1_score(test_df['status'], y_test_predicted)

print(f1)

0.0


Очевидно, что модель плохая, возможно потому, что у нас мало данных, особенно показателей суицидальных пациентов. 